Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [ ]:
NAME = "Ameya"
COLLABORATORS = ""

---

This notebook will show you an example of how to connect a simulation that's running on google colab with a pymol session running on your computer. What's happening here is that structures are being sent from colab to a server at Johns Hopkins, and then your pymol session is asking for sturctures from Hopkins. The way you end up with the right structures is by having a password / secret phrase that identifies the structures as yours. Let's take a look. 

In [ ]:
import sys
if 'google.colab' in sys.modules:
  !pip install pyrosettacolabsetup
  import pyrosettacolabsetup
  pyrosettacolabsetup.mount_pyrosetta_install()
from pyrosetta import *
init()


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Mounted at /content/google_drive
Looking for compatible PyRosetta wheel file at google-drive/PyRosetta/colab.bin/wheels...
Found compatible wheel: /content/google_drive/MyDrive/PyRosetta/colab.bin/wheels//content/google_drive/MyDrive/PyRosetta/colab.bin/wheels/pyrosetta-2022.28+release.6c2ddeccd31-cp37-cp37m-linux_x86_64.whl
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


PyRosetta-4 2022 [Rosetta PyRosetta4.MinSizeRel.python37.ubuntu 2022.28+release.6c2ddeccd31e780da01b0cbbf3e33949218c99d9 2022-07-12T15:12:13] retrieved from: http://www.pyrosetta.org
(C) Copyright Rosetta Commons Member Institutions. Created in JHU by Sergey Lyskov and PyRosetta Team.
core.init: Checking for fconfig files in pwd and ./rosetta/flags
core.init: Rosetta version: PyRosetta4.MinSizeRel.python37.ubuntu r324 2022.28+release.6c2ddeccd31 6c2ddeccd31e780da01b0cb

Ok, so, in order to use the PyMOLMover and have it send your structures to Hopkins, you need to use this cell below. You will need to use a cell like this any time you wish to send structures over the wire to a PyMOL session you have running locally. In this cell, you're going to enter your password-like secret phrase. 

You should also have downloaded the PyMOL-Rosetta-relay-client.python3.py script and put it in a useful location (e.g. `~/pymol_scripts/`), put your secret phrase into this script (replacing the `_my_secret_ = None` line with `_my_secret_ = "thesecretikeep"` or whtever phrase you used), and then edited your `.pymolrc` file in your home directory (or from within PyMOL) so that it runs this script every time you fire up pymol (e.g. `run /Users/doug/pymol_scripts/PyMOL-Rosetta-relay-client.python3.py` if `/Users/doug` is the path to your home directory, as it is for me).

Go ahead and open up a PyMOL session. If it's working correctly, it should print something like
```
PyMOL>run /Users/doug/pymol_scripts/PyMOL-Rosetta-relay-client.python3.py
WARNING: CONNECTION to relay server relay.graylab.jhu.edu:9989 IS NOT ENCRYPTED!
Connected to relay.graylab.jhu.edu:9989...
```

In [ ]:
if os.getenv("DEBUG"): sys.exit(0)
import pyrosetta.network

# make sure to supply unique secret here and use _the same_ secrete when running PyMOL-Rosetta-relay-client.python3.py in PyMOL on your desktop machine
pyrosetta.network.start_udp_to_tcp_bridge_daemon(secret='ROSETTA_PYMOL_SECRET')

# note that you can use random numbers to generate _good_ unique secret, like this: 
# import random, string; print( "".join( random.sample( string.ascii_letters + string.digits, 16 ) ) )


UDP server started: localhost:65000...
Connected to relay.graylab.jhu.edu 128.220.208.35:9989...


Ok, now you're ready to create a PyMOL Observer and start watching a protocol alter a Pose. Make sure PyMOL is open before you run the cell below.

Note that you might very well want the cell to stop running before it hits 100 iterations.

In [ ]:
import time as time_module

import pyrosetta
import pyrosetta.rosetta as rosetta

pose = pyrosetta.pose_from_sequence('SERGEY'*2)
rosetta.protocols.moves.AddPyMOLObserver(pose, keep_history=False)

scorefxn = pyrosetta.get_fa_scorefxn() #  rosetta.create_score_function('standard')

psi = 0.0
phi = 0.0
i = 1
while i < 100:
    pose.set_psi(i, pose.psi(i) + 87)
    pose.set_phi(i, pose.phi(i) + 19)
    scorefxn(pose)

    time_module.sleep(.05)
    i += 1
    if i >pose.size(): i=1


core.chemical.GlobalResidueTypeSet: Finished initializing fa_standard residue type set.  Created 985 residue types
core.chemical.GlobalResidueTypeSet: Total time to initialize 0.954696 seconds.
core.scoring.ScoreFunctionFactory: SCOREFUNCTION: ref2015
core.scoring.etable: Starting energy table calculation
core.scoring.etable: smooth_etable: changing atr/rep split to bottom of energy well
core.scoring.etable: smooth_etable: spline smoothing lj etables (maxdis = 6)
core.scoring.etable: smooth_etable: spline smoothing solvation etables (max_dis = 6)
core.scoring.etable: Finished calculating energy tables.
basic.io.database: Database file opened: scoring/score_functions/hbonds/ref2015_params/HBPoly1D.csv
basic.io.database: Database file opened: scoring/score_functions/hbonds/ref2015_params/HBFadeIntervals.csv
basic.io.database: Database file opened: scoring/score_functions/hbonds/ref2015_params/HBEval.csv
basic.io.database: Database file opened: scoring/score_functions/hbonds/ref2015_param